In [22]:
import numpy as np
import torch
import torch.nn as nn

from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import torch.nn.functional as F

siapkan fitur dan target, jangan lupa ubah ke tensor

In [2]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

fungsi library TensorDataset adalah agar dapat mengakses baris dari inputs dan targets sebagai tuples. Juga menyediakan
standard API yang dapat digunakan di banyak jenis dataset dalam pytorch

In [6]:
# definisikan dataset
train_ds = TensorDataset(inputs, targets)
# ambil 3 baris pertama dari dataset
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

fungsi library DataLoader adalah untuk memisahkan data menjadi beberapa bagian dengan besaran porsi yang telah ditentukan. Dapat digunakan untuk memisahkan antara data latih dengan data uji

In [11]:
# definisikan data loader
batch_size = 5
# ukuran data yang dipisahkan adalah 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [12]:
# intip isi train_dl
for xb, yb in train_dl :
    print(xb)
    print(yb)
    break

tensor([[ 87., 134.,  58.],
        [103.,  43.,  36.],
        [ 73.,  67.,  43.],
        [ 92.,  87.,  64.],
        [ 91.,  88.,  64.]])
tensor([[119., 133.],
        [ 20.,  38.],
        [ 56.,  70.],
        [ 82., 100.],
        [ 81., 101.]])


selanjutnya, daripada menginisialisasikan weight dan bias. Disini pakai nn.Linear untuk memodelkan linear regression secara otomatis

In [16]:
# definisikan model
model = nn.Linear(3,2)
# 3 berarti jumlah fitur yang menjadi input adalah 3
# 2 berarti jumlah target

print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.4304, -0.3432,  0.4476],
        [ 0.2356, -0.3945,  0.1522]], requires_grad=True)
Parameter containing:
tensor([0.5332, 0.2782], requires_grad=True)


In [18]:
# intip parameter sebuah model
model.parameters

<bound method Module.parameters of Linear(in_features=3, out_features=2, bias=True)>

In [19]:
# list parameter buat liat weight dan bias
list(model.parameters())

[Parameter containing:
 tensor([[-0.4304, -0.3432,  0.4476],
         [ 0.2356, -0.3945,  0.1522]], requires_grad=True),
 Parameter containing:
 tensor([0.5332, 0.2782], requires_grad=True)]

In [20]:
# cara lakukan prediksi
preds = model(inputs)
preds

tensor([[-34.6304,  -2.4147],
        [-40.1843,  -3.2635],
        [-56.9356, -23.2682],
        [-41.5599,  12.9726],
        [-30.7762, -10.6892],
        [-34.7176,  -1.7846],
        [-39.3935,  -2.7168],
        [-56.9183, -22.8804],
        [-41.4728,  12.3425],
        [-29.8983, -10.7725],
        [-33.8396,  -1.8680],
        [-40.2715,  -2.6334],
        [-57.7264, -23.8149],
        [-42.4379,  13.0559],
        [-30.6891, -11.3193]], grad_fn=<AddmmBackward0>)

## Loss Function
pakai library torch.nn.functional untuk dapetin loss function

In [23]:
# definisikan loss function
loss_fn = F.mse_loss

hitung loss dari prediksi dengan model sebelumnya

In [24]:
# preds = model(inputs)
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(13331.1572, grad_fn=<MseLossBackward0>)


bisa dilihat bahwa mse masih sangat besar

In [25]:
# untuk tau asal muasal dan fungsi dari F.mse_Loss
?F.mse_loss

## Optimizer
Di bab sebelumnya, kita memanipulasi weigth dan bias pada model menggunakan gradient. Namun sekarang cukup menggunakan optimizer optim.SGD. SGD artinya adalah Stochastic Gradient Descent. Dinamakan Stohastic karena sampel dipilih dalam sebagian porsi (biasanya random), bukan berupa sebuah grup

In [26]:
# definisikan optimizer
opt = torch.optim.SGD(model.parameters(),lr=1e-5)

model.parameter harus diberikan kepada optimizer, agar optimizer tahu matrix mana yang harus dimodifikasi. Juga kita dapat mendefinisikan learning rate (lr) yang akan mengontrol seberapa besar modifikasi parameter yang akan dilakukan

## Melakukan Pelatihan Model
Setelah mendapatkan model dan optimizer. Sekarang waktunya melakukan training data. Adapun step-step nya seperti ini :
1. melakukan prediksi
2. hitung loss
3. hitung gradient w.r.t serta weight dan bias
4. mengatur ulang weight dengan cara menguranginya dengan sedikit nilai
5. reset gradient ke 0

bedanya adalah, disini hanya memakai sebagian data, bukan keseluruhan data dalam tiap iterasi. Pakai fungsi fit untuk melakukan train pada model sebanyak epoch yang telah ditentukan

In [58]:
# utility function untuk train model
def fit(num_epochs, model, loss_fn, opt, train_dl) :
    
    #ulangi sebanyak angka yang diberikan
    for epoch in range(num_epochs) :
        
        #lakukan train batch data
        for xb,yb in train_dl :
        
            #1. lakukan prediksi
            pred = model(xb)
        
            #2. hitung loss
            loss = loss_fn(pred, yb)
        
            #3. hitung gradient
            loss.backward()
        
            #4. atur ulang parameter weight pakai gradient
            opt.step()
        
            #5. reset gradient ke 0
            opt.zero_grad()
        
        #print progress
        if (epoch+1) % 10 == 0 :
            print('Epoch[{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [59]:
fit(100, model, loss_fn, opt, train_dl)

Epoch[10/100], Loss: 1037.7046
Epoch[20/100], Loss: 243.4281
Epoch[30/100], Loss: 251.7603
Epoch[40/100], Loss: 205.2507
Epoch[50/100], Loss: 109.1266
Epoch[60/100], Loss: 90.6197
Epoch[70/100], Loss: 74.5982
Epoch[80/100], Loss: 95.3650
Epoch[90/100], Loss: 47.0768
Epoch[100/100], Loss: 24.0847


In [60]:
preds = model(inputs)
preds

tensor([[ 58.4197,  71.8489],
        [ 82.7211, 100.8026],
        [114.5305, 130.0701],
        [ 28.5758,  45.6674],
        [ 98.6219, 114.3873],
        [ 57.3937,  70.9816],
        [ 82.6111, 100.8532],
        [114.8972, 130.7035],
        [ 29.6017,  46.5347],
        [ 99.5379, 115.3053],
        [ 58.3098,  71.8996],
        [ 81.6951,  99.9353],
        [114.6405, 130.0195],
        [ 27.6598,  44.7494],
        [ 99.6478, 115.2547]], grad_fn=<AddmmBackward0>)

In [61]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])

Dapat dilihat bahwa setelah dilakukan training sebanyak 100 kali, terdapat perubahan signifikan pada mse. selisih antara prediksi dengan target juga tidak terlalu berbeda jauh